In [265]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam

import seaborn as sns
sns.set()

df = pd.read_csv('D:/IITM/DeepCache/syntheticDataset_O50.csv')
df.head()

,object_ID,request_time
0,5,2.961472
1,25,3.274127
2,2,3.785475
3,2,4.455687
4,4,5.288994


In [266]:
data = df[df.columns[0]].values
len(data)/2

146073.0

In [267]:
latent_dim = 128 # LSTM hidden units
dropout = .20 


In [268]:
encoder_inputs = Input(shape=(None, 1)) 
encoder = LSTM(latent_dim, dropout=dropout, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [269]:
decoder_inputs = Input(shape=(None, 1)) 
decoder_lstm = LSTM(latent_dim, dropout=dropout, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)


In [270]:
decoder_dense = Dense(1) # 1 continuous output at each timestep
decoder_outputs = decoder_dense(decoder_outputs)

In [271]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [272]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, None, 1)      0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, None, 1)      0                                            
__________________________________________________________________________________________________
lstm_31 (LSTM)                  [(None, 128), (None, 66560       input_33[0][0]                   
__________________________________________________________________________________________________
lstm_32 (LSTM)                  [(None, None, 128),  66560       input_34[0][0]                   
                                                                 lstm_31[0][1]                    
          

In [273]:
batch_size = 25000
epochs = 5



In [274]:
decoder_target_data = data[200000:200100]
decoder_target_data = decoder_target_data.reshape((10,10,1))
encoder_input_data = data[0:200000]
encoder_input_data = encoder_input_data.reshape(decoder_input_data.shape[0],20000,1)

print(encoder_input_data.shape,decoder_target_data.shape)

(10, 20000, 1) (10, 10, 1)


In [275]:
decoder_input_data = np.zeros(decoder_target_data.shape)
decoder_input_data[1:,0,0] = decoder_target_data[:-1,0,0]
decoder_input_data[0,0,0] = encoder_input_data[-1,0,0]
decoder_input_data.shape

(10, 10, 1)

In [ ]:
model.compile(Adam(), loss='mean_absolute_error')
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Train on 8 samples, validate on 2 samples
Epoch 1/5
8/8 [==============================] - 632s 79s/step - loss: 25.9923 - val_loss: nan
Epoch 2/5
8/8 [==============================] - 635s 79s/step - loss: nan - val_loss: nan
Epoch 3/5


In [246]:
encoder_model = Model(encoder_inputs, encoder_states)

# A modified version of the decoding stage that takes in predicted target inputs
# and encoded state vectors, returning predicted target outputs and decoder state vectors.
# We need to hang onto these state vectors to run the next step of the inference loop.
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

def decode_sequence(input_seq):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 1))
    
    # Populate the first target sequence with end of encoding series pageviews
    target_seq[0, 0, 0] = input_seq[0, -1, 0]

    # Sampling loop for a batch of sequences - we will fill decoded_seq with predictions
    # (to simplify, here we assume a batch of size 1).

    decoded_seq = np.zeros((1,pred_steps,1))
    for i in range(pred_steps):
        
        output, h, c = decoder_model.predict([target_seq] + states_value)
        
        decoded_seq[0,i,0] = output[0,0,0]

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, 1))
        target_seq[0, 0, 0] = output[0,0,0]

        # Update states
        states_value = [h, c]

    return decoded_seq

In [250]:
p = decode_sequence(encoder_input_data)

NameError: name 'pred_steps' is not defined

In [ ]:
p